In [13]:
# import os
# import random
# import time
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# import time

# def crawl_site(start_url, visited_urls_file, urls_file, not_visiting_urls_file):
    
#     # 初始化Options
#     options = Options()
#     options.add_argument('--headless')  # 无头模式
#     options.add_argument('--disable-gpu')
#     options.add_argument('--window-size=1920x1080')  # 设置窗口大小，这对某些页面布局很重要
#     options.add_argument('--disable-extensions')
#     options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
#     options.add_argument('--no-sandbox')  # Bypass OS security model

#     # 初始化Chrome WebDriver
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
#     # 初始化集合
#     not_visiting_urls = set()
#     visited_urls = set()
#     urls_to_visit = set([start_url])

#     # 在函数的开头打开文件，并在整个函数执行期间保持打开状态
#     with open(urls_file, 'a+', encoding='utf-8') as urls_f, \
#         open(visited_urls_file, 'a+', encoding='utf-8') as visited_urls_f, \
#         open(not_visiting_urls_file, 'r', encoding='utf-8') as not_visiting_urls_f:
#         # 读取 urls_file 文件并更新 urls_to_visit 集合
#         urls_f.seek(0)
#         for url in urls_f:
#             urls_to_visit.add(url.strip())

#         # 读取 visited_urls_file 文件并更新 visited_urls 集合
#         visited_urls_f.seek(0)
#         for url in visited_urls_f:
#             visited_urls.add(url.strip())
            
#         not_visiting_urls_f.seek(0)
#         for url in not_visiting_urls_f:
#             not_visiting_urls.add(url.strip())

#         # 从 urls_to_visit 中去除已访问过的 URLs
#         urls_to_visit.difference_update(visited_urls)

#         while urls_to_visit:
#             current_url = urls_to_visit.pop()

#             # 只有当current_url不在visited_urls里时才进行爬取
#             if current_url not in visited_urls and all(sub_url not in current_url for sub_url in not_visiting_urls):
#                 print(f"Visiting: {current_url}")
#                 # 在尝试访问链接前，添加随机延迟
#                 time.sleep(random.randint(1, 3))

#                 try:
#                     # response = requests.get(current_url)
#                     # soup = BeautifulSoup(response.text, 'html.parser')
#                     driver.get(current_url)
#                     soup = BeautifulSoup(driver.page_source, 'html.parser')
                    

#                     # 提取并存储所有文字内容
#                     text_content = soup.get_text()

#                     # 移除current_url中的"https:"，并为文件路径准备
#                     sanitized_url = current_url.replace('https://', '').replace('http://', '')
#                     content_directory = f"scraping_data/{sanitized_url}/"
#                     content_file_path = os.path.join(content_directory, "content.txt") #加个scraping_data
                    
#                     # 找到所有的动态加载的div元素
#                     dynamic_divs = driver.find_elements(By.CSS_SELECTOR, 'div.js-card-gallery')
                    
#                     # 确保目录存在
#                     os.makedirs(content_directory, exist_ok=True)

#                     with open(content_file_path, 'w', encoding='utf-8') as content_f:
#                         content_f.write(f"URL: {current_url}\n{text_content}\n{'='*100}\n\n")

#                         # 遍历每个div并写入它的文本内容
#                         for div in dynamic_divs:
#                             content_f.write(f"{div.text}\n")  # 写入div的文本内容
                    
#                     if "cis" in current_url:
                            
#                         # 查找并处理所有链接
#                         for link in soup.find_all('a', href=True):
#                             absolute_link = urljoin(current_url, link['href'])
#                             if (absolute_link not in visited_urls) and \
#                             (absolute_link not in urls_to_visit) and \
#                             all(sub_url not in absolute_link for sub_url in not_visiting_urls):
#                                 urls_to_visit.add(absolute_link)
#                                 urls_f.write(f"{absolute_link}\n")
                    
#                         for div in dynamic_divs:
#                             # 在每个div中查找所有的<a>标签
#                             links = div.find_elements(By.TAG_NAME, 'a')
#                             # 输出每个链接的href属性
#                             for link in links:
#                                 div_url = link.get_attribute('href')
#                                 if (absolute_link not in visited_urls) and \
#                                 (absolute_link not in urls_to_visit) and \
#                                 all(sub_url not in absolute_link for sub_url in not_visiting_urls):
#                                     urls_to_visit.add(div_url)
#                                     urls_f.write(f"{div_url}\n")


#                         visited_urls.add(current_url)
#                         visited_urls_f.write(f"{current_url}\n")
#                 except requests.RequestException as e:
#                     print(f"Error during requests to {current_url}: {str(e)}")


# if __name__ == "__main__":
#     start_url = 'https://cis.unimelb.edu.au/'
#     visited_urls_file = 'scraping_urls/visited_urls.txt'
#     urls_file = 'scraping_urls/urls.txt'
#     not_visiting_urls_file = 'scraping_urls/not_visiting_urls.txt'
#     crawl_site(start_url, visited_urls_file, urls_file, not_visiting_urls_file)

In [14]:
import os
import random
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

def crawl_site(start_url, visited_urls_file, urls_file, not_visiting_urls_file):
    
    # 初始化Options
    options = Options()
    options.add_argument('--headless')  # 无头模式
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')  # 设置窗口大小，这对某些页面布局很重要
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    options.add_argument('--no-sandbox')  # Bypass OS security model
    
    # 设置模拟用户的请求头
    user_agent = (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/104.0.5112.79 Safari/537.36"
    )
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument('accept-language=en-US,en;q=0.9')
    options.add_argument('accept-encoding=gzip, deflate, br')
    options.add_argument('referer=https://www.google.com')
    
    # 初始化Chrome WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # 初始化集合
    not_visiting_urls = set()
    visited_urls = set()
    urls_to_visit = set([start_url])

    # 在函数的开头打开文件，并在整个函数执行期间保持打开状态
    with open(urls_file, 'a+', encoding='utf-8') as urls_f, \
        open(visited_urls_file, 'a+', encoding='utf-8') as visited_urls_f, \
        open(not_visiting_urls_file, 'r', encoding='utf-8') as not_visiting_urls_f:
        # 读取 urls_file 文件并更新 urls_to_visit 集合
        urls_f.seek(0)
        for url in urls_f:
            urls_to_visit.add(url.strip())

        # 读取 visited_urls_file 文件并更新 visited_urls 集合
        visited_urls_f.seek(0)
        for url in visited_urls_f:
            visited_urls.add(url.strip())
            
        not_visiting_urls_f.seek(0)
        for url in not_visiting_urls_f:
            not_visiting_urls.add(url.strip())

        # 从 urls_to_visit 中去除已访问过的 URLs
        urls_to_visit.difference_update(visited_urls)

        while urls_to_visit:
            current_url = urls_to_visit.pop()

            # 只有当current_url不在visited_urls里时才进行爬取
            if current_url not in visited_urls and all(sub_url not in current_url for sub_url in not_visiting_urls) \
                and "unimelb" in current_url \
                and "linkedin" not in current_url:
                print(f"Visiting: {current_url}")
                # 在尝试访问链接前，添加随机延迟
                time.sleep(random.randint(1, 3))

                try:
                    # 访问当前URL并解析页面内容
                    driver.get(current_url)
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                    # 提取并存储所有文字内容
                    text_content = soup.get_text()

                    # 移除current_url中的"https:"，并为文件路径准备
                    sanitized_url = current_url.replace('https://', '').replace('http://', '')
                    content_directory = f"scraping_data/{sanitized_url}/"
                    content_file_path = os.path.join(content_directory, "content.txt") #加个scraping_data
                    
                    # 找到所有的动态加载的div元素
                    dynamic_divs = driver.find_elements(By.CSS_SELECTOR, 'div.js-card-gallery')
                    
                    # 确保目录存在
                    os.makedirs(content_directory, exist_ok=True)

                    with open(content_file_path, 'w', encoding='utf-8') as content_f:
                        content_f.write(f"URL: {current_url}\n{text_content}\n{'='*100}\n\n")

                        # 遍历每个div并写入它的文本内容
                        for div in dynamic_divs:
                            content_f.write(f"{div.text}\n")  # 写入div的文本内容
                    
                    if "cis" in current_url:
                            
                        # 查找并处理所有链接
                        for link in soup.find_all('a', href=True):
                            absolute_link = urljoin(current_url, link['href'])
                            if (absolute_link not in visited_urls) and \
                            (absolute_link not in urls_to_visit) and \
                            ("unimelb" in absolute_link) and \
                            all(sub_url not in absolute_link for sub_url in not_visiting_urls):
                                urls_to_visit.add(absolute_link)
                                urls_f.write(f"{absolute_link}\n")
                    
                        for div in dynamic_divs:
                            # 在每个div中查找所有的<a>标签
                            links = div.find_elements(By.TAG_NAME, 'a')
                            # 输出每个链接的href属性
                            for link in links:
                                div_url = link.get_attribute('href')
                                if (div_url not in visited_urls) and \
                                (div_url not in urls_to_visit) and \
                                ("unimelb" in div_url) and \
                                all(sub_url not in div_url for sub_url in not_visiting_urls):
                                    urls_to_visit.add(div_url)
                                    urls_f.write(f"{div_url}\n")


                    visited_urls.add(current_url)
                    visited_urls_f.write(f"{current_url}\n")
                except WebDriverException as e:
                    # Catch specific errors like ERR_NAME_NOT_RESOLVED and move on to the next URL
                    if "ERR_NAME_NOT_RESOLVED" in str(e):
                        print(f"Skipping URL due to error: {current_url} - {str(e)}")
                        continue
                    else:
                        print(f"Error during requests to {current_url}: {str(e)}")
                        continue

if __name__ == "__main__":
    start_url = 'https://cis.unimelb.edu.au/'
    visited_urls_file = 'scraping_urls/visited_urls.txt'
    urls_file = 'scraping_urls/urls.txt'
    not_visiting_urls_file = 'scraping_urls/not_visiting_urls.txt'
    crawl_site(start_url, visited_urls_file, urls_file, not_visiting_urls_file)
